# [`wordplay` 🎮 💬](https://github.com/saforem2/wordplay): Shakespeare

✍️

[Sam Foreman](https://samforeman.me)
(\[[ALCF](https://alcf.anl.gov/about/people/sam-foreman)\](<https://alcf.anl.gov/about/people/sam-foreman>))  
2025-07-22

We will be using the [Shakespeare
dataset](https://github.com/saforem2/wordplay/blob/main/data/shakespeare/readme.md)
to train a (~ small) 10M param LLM *from scratch*.

<img src="https://github.com/saforem2/wordplay/blob/main/assets/shakespeare.jpeg?raw=true" width="45%" align="center" /><br>

Image generated from
[stabilityai/stable-diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
on [🤗 Spaces](https://huggingface.co/spaces).<br>

<details closed>

<summary>

Prompt Details

</summary>

<ul>

<li>

Prompt:

</li>

<t><q> Shakespeare himself, dressed in full Shakespearean garb, writing
code at a modern workstation with multiple monitors, hacking away
profusely, backlit, high quality for publication </q></t>

<li>

Negative Prompt:

</li>

<t><q> low quality, 3d, photorealistic, ugly </q></t>

</ul>

</details>

## Install / Setup

<b>Warning!</b><br>

**IF YOU ARE EXECUTING ON GOOGLE COLAB**:

You will need to restart your runtime (`Runtime` $\rightarrow\,$
`Restart runtime`)  
*after* executing the following cell:

In [2]:
%%bash

python3 -c 'import wordplay; print(wordplay.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has wordplay installed. Nothing to do."
else
    echo "Does not have wordplay installed. Installing..."
    git clone 'https://github.com/saforem2/wordplay'
    python3 wordplay/data/shakespeare_char/prepare.py
    python3 wordplay/data/shakespeare/prepare.py
    python3 -m pip install deepspeed
    python3 -m pip install -e wordplay
fi

/Users/samforeman/projects/saforem2/wordplay/src/wordplay/__init__.py
Has wordplay installed. Nothing to do.

## Post Install

If installed correctly, you should be able to:

``` python
>>> import wordplay
>>> wordplay.__file__
'/path/to/wordplay/src/wordplay/__init__.py'
```

In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
import ezpz

os.environ['COLORTERM'] = 'truecolor'
if sys.platform == 'darwin':
    # If running on MacOS:
    # os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
    os.environ['TORCH_DEVICE'] = 'cpu'
# -----------------------------------------------

logger = ezpz.get_logger()

import wordplay
logger.info(wordplay.__file__)

[ 2025-07-23 08:57:08,682995 ][ I ][ ezpz / __init__ : 265 : ezpz ] Setting logging level to 'INFO' on 'RANK == 0'

[ 2025-07-23 08:57:08,685371 ][ I ][ ezpz / __init__ : 266 : ezpz ] Setting logging level to 'CRITICAL' on all others 'RANK != 0'

[ 2025-07-23 08:57:08,688375 ][ I ][ ipykernel_34035 / 2338663768 : 17 : ezpz.log ] /Users/samforeman/projects/saforem2/wordplay/src/wordplay/ __init__.py

## Build Trainer

Explicitly, we:

1.  `setup_torch(...)`
2.  Build `cfg: DictConfig = get_config(...)`
3.  Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4.  Build `trainer = Trainer(config)`

In [4]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

BACKEND = 'DDP'

rank = setup(
    framework='pytorch',
    backend=BACKEND,
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'model.batch_size=1',
        'model.block_size=128',
        'optimizer=shakespeare',
        'train=shakespeare',
        f'train.backend={BACKEND}',
        'train.compile=false',
        'train.dtype=bfloat16',
        'train.max_iters=500',
        'train.log_interval=10',
        'train.eval_interval=50',
    ]
)
config = instantiate(cfg)

[ 2025-07-23 08:57:08,791647 ][ I ][ wordplay / configs : 81 ] Setting HF_DATASETS_CACHE to /Users/samforeman/projects/saforem2/wordplay/.cache/huggingface/ datasets

[ 2025-07-23 08:57:08,798661 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-23 08:57:08,799625 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-23 08:57:08,800251 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-23 08:57:08,800836 ][ I ][ ezpz / dist : 1159 ] Using fw = 'ddp' with torch_ { device,backend } = { cpu, gloo }

...

[ 2025-07-23 08:57:08,930199 ][ I ][ wordplay / configs : 317 ] Loading val from /Users/samforeman/projects/saforem2/wordplay/data/shakespeare_char/ val.bin

[ 2025-07-23 08:57:08,931696 ][ I ][ wordplay / configs : 442 ] Tokens per iteration: 128

[ 2025-07-23 08:57:08,932528 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-23 08:57:08,933099 ][ I ][ wordplay / configs : 465 ] Using self. ptdtype = torch .bfloat16 on self. device_type = 'cpu'

[ 2025-07-23 08:57:08,933802 ][ I ][ wordplay / configs : 471 ] Initializing a new model from scratch

### Build `Trainer` object

In [5]:
trainer = Trainer(config)

[ 2025-07-23 08:57:08,965527 ][ I ][ wordplay / trainer : 235 ] Initializing a new model from scratch

[ 2025-07-23 08:57:09,139070 ][ I ][ wordplay / model : 255 ] number of parameters: 10. 65M

[ 2025-07-23 08:57:09,140622 ][ I ][ wordplay / trainer : 252 ] Model size: num_params = 10646784

[ 2025-07-23 08:57:09,141366 ][ I ][ wordplay / model : 445 ] num decayed parameter tensors: 26 , with 10 , 690 , 944 parameters

[ 2025-07-23 08:57:09,141947 ][ I ][ wordplay / model : 449 ] num non-decayed parameter tensors: 13 , with 4 , 992 parameters

...

[ 2025-07-23 08:57:09,143723 ][ C ][ wordplay / trainer : 308 ] "devid='cpu:0'"

[ 2025-07-23 08:57:09,151176 ][ I ][ wordplay / trainer : 347 ] • self. model = GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 128 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

[ 2025-07-23 08:57:09,154125 ][ I ][ wordplay / trainer : 348 ] • self. grad_scaler = None

[ 2025-07-23 08:57:09,154879 ][ I ][ wordplay / trainer : 349 ] • self. model_engine = GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 128 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

[ 2025-07-23 08:57:09,157754 ][ I ][ wordplay / trainer : 350 ] • self. optimizer = AdamW ( 
Parameter Group 0 
 amsgrad: False 
 betas: ( 0.9 , 0.99 ) 
 capturable: False 
 decoupled_weight_decay: True 
 differentiable: False 
 eps: 1e-08 
 foreach: None 
 fused: None 
 lr: 0.001 
 maximize: False 
 weight_decay: 0.1 

Parameter Group 1 
 amsgrad: False 
 betas: ( 0.9 , 0.99 ) 
 capturable: False 
 decoupled_weight_decay: True 
 differentiable: False 
 eps: 1e-08 
 foreach: None 
 fused: None 
 lr: 0.001 
 maximize: False 
 weight_decay: 0.0 
 )

## Prompt (**prior** to training)

In [6]:
query = "What is an LLM?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-23 08:57:11,482806 ][ I ][ ipykernel_34035 / 3496000222 : 9 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 08:57:11,483979 ][ I ][ ipykernel_34035 / 3496000222 : 10 : ezpz.log ] [ 'response' ] :

What is an LLM?A,,osy'exx.ff.fpppxv;;'vt3QjYhhvvYAhowQwwQ,eqeqG;X.YqqQSZQWLsyccccj:ZhaooxkkcfkZ
ffop- f,hqWl
oocpppUqAQ;cc' 'bQqcWAttrqerrwyqqsrqttqYeqWQs' tottcqestbqbbrpWbWYAppppBqfhcqqYqqM?qttqQU'gYe?A.. 'S' rtppW'fJf;??qn.pwrrrqqfA;!!A,,,AtqqqqbW;bSoW;;?;;;qQ;;cIA. 'M;' 'g

## Train Model

|  name  |        description         |
|:------:|:--------------------------:|
| `step` |   Current training step    |
| `loss` |         Loss value         |
|  `dt`  | Time per step (in **ms**)  |
| `sps`  |     Samples per second     |
| `mtps` |  (million) Tokens per sec  |
| `mfu`  | Model Flops utilization[1] |

^legend: #tbl-legend

[1] in units of A100 `bfloat16` peak FLOPS

In [7]:
trainer.config.device_type

'cpu'

In [8]:
from rich import print

print(trainer.model)

GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 128 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

## (partial) Training:

We’ll first train for 500 iterations and then evaluate the models
performance on the same prompt:

> What is an LLM?

In [9]:
trainer.train(train_iters=500)

Training Legend 
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ abbr ┃ desc ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ step │ Current training iteration │
│ loss │ Loss value │
│ dt │ Elapsed time per training step │
│ dtf │ Elapsed time per forward step │
│ dtb │ Elapsed time per backward step │
│ sps │ Samples per second │
│ sps_per_gpu │ Samples per second (per GPU) │
│ tps │ Tokens per second │
│ tps_per_gpu │ Tokens per second (per GPU) │
│ mfu │ Model flops utilization │
└─────────────┴────────────────────────────────┘

[ 2025-07-23 08:57:13,916810 ][ I ][ wordplay / trainer : 790 ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 08:57:13,917750 ][ I ][ wordplay / trainer : 794 ] [ 'response' ] :

What is an LLM?wCw'.AAAfxo.. 'yfAQfppyybvFYerr.MfYZAcLyQQCkkexx-3lllrpMqxkko-rZx3b' 3j -ffSSoqq3hhdf'Q' 'aq' wqqsoKZb'ec3ZAAA;;o,qff.. 'fArttgbYtturcbcSYrS-Fff' wwwerwPgJ;.e;yY-SpuyeexqYqgQtpMSYqYgbtQqq' '' ;pfsw, ';oA;qqeqcckSAo,,rooMgyQha''' fAA..gg;;'ggtSvrupptkeweqqcqqkk-SvYYIv

[ 2025-07-23 08:57:18,166363 ][ I ][ wordplay / trainer : 850 ] step = 10 loss = 4.28757 dt = 0.0180821 dtf = 0.0177301 dtb = 0.000160959 sps = 55.3034 sps_per_gpu = 55.3034 tps = 7078.83 tps_per_gpu = 7078.83 mfu = 0.152965

[ 2025-07-23 08:57:18,353237 ][ I ][ wordplay / trainer : 850 ] step = 20 loss = 4.28569 dt = 0.0179544 dtf = 0.0175802 dtb = 0.000170584 sps = 55.6966 sps_per_gpu = 55.6966 tps = 7129.16 tps_per_gpu = 7129.16 mfu = 0.153074

...

[ 2025-07-23 08:58:16,813979 ][ I ][ wordplay / trainer : 850 ] step = 460 loss = 4.24269 dt = 0.0198506 dtf = 0.0193689 dtb = 0.0001835 sps = 50.3762 sps_per_gpu = 50.3762 tps = 6448.16 tps_per_gpu = 6448.16 mfu = 0.13305

[ 2025-07-23 08:58:17,027692 ][ I ][ wordplay / trainer : 850 ] step = 470 loss = 4.26877 dt = 0.0205835 dtf = 0.019941 dtb = 0.00031125 sps = 48.5826 sps_per_gpu = 48.5826 tps = 6218.57 tps_per_gpu = 6218.57 mfu = 0.133183

[ 2025-07-23 08:58:17,225952 ][ I ][ wordplay / trainer : 850 ] step = 480 loss = 4.19188 dt = 0.0193338 dtf = 0.0186521 dtb = 0.00012275 sps = 51.7228 sps_per_gpu = 51.7228 tps = 6620.52 tps_per_gpu = 6620.52 mfu = 0.134171

[ 2025-07-23 08:58:17,430493 ][ I ][ wordplay / trainer : 850 ] step = 490 loss = 4.22611 dt = 0.02165 dtf = 0.0212389 dtb = 0.000145625 sps = 46.1894 sps_per_gpu = 46.1894 tps = 5912.24 tps_per_gpu = 5912.24 mfu = 0.133529

[ 2025-07-23 08:58:17,639397 ][ I ][ wordplay / trainer : 850 ] step = 500 loss = 4.21804 dt = 0.0192383 dtf = 0.018851 dtb = 0.000150875 sps = 51.9796 sps_per_gpu = 51.9796 tps = 6653.38 tps_per_gpu = 6653.38 mfu = 0.134554

In [10]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-23 08:58:19,743301 ][ I ][ ipykernel_34035 / 1425179755 : 12 : ezpz.log ] took: 1. 9168s

[ 2025-07-23 08:58:19,745116 ][ I ][ ipykernel_34035 / 1425179755 : 13 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 08:58:19,745939 ][ I ][ ipykernel_34035 / 1425179755 : 14 : ezpz.log ] [ 'response' ] :

What is an LLM?fwll

b3afqbZZI,r oppq3A33QoUUye-fwC'3b3. ',A' .hhPlVXXqeQyCCC;xfssc;wTTTTcdGoeehQOCXXXB'KZ--qehoF3AqfqqW
cQAcceffGG, 'fSJpppww,txMgQs;;;?qf' fSSrpcg?s,A'rr,aso? '' o'MtQrrSSgqfttggSc' 'Wb' qA,.Apcbb???;pYYySQ'agggScWQgbqWfqYroffSYSYhqfk' 'qfAA,sgWlnZ:pt,JynS' gJZes

## Resume Training…

In [11]:
trainer.train()

[ 2025-07-23 08:58:21,718373 ][ I ][ wordplay / trainer : 790 ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 08:58:21,720303 ][ I ][ wordplay / trainer : 794 ] [ 'response' ] :

What is an LLM?wZbbbT'3weew, 'foBB.qWWlpwes.qqQevFAA.bbvFF-AkacWWfYhx3fooB''' ';vveesppWW
eeWA3ZZppPZe;dCCvres ;ecc--Ws'cqor,JZVVVCCeepfqqWxApBBBBhh;;JeQhMMss,,wshrhW?BiMWYqqwwwAASSwrrroo,rqtWseMq.Ak'ofA,, 't,,..hh;xx' ?sAq';cqxrqWkeMqt'gzAAxhrpqt'g't;?btoseq-pqq'qAtttt,eqrM

[ 2025-07-23 08:58:25,226804 ][ I ][ wordplay / trainer : 733 ] Saving checkpoint to: /Users/samforeman/projects/saforem2/intro-hpc-bootcamp-2025/content/02-llms/ 7-shakespeare-example

[ 2025-07-23 08:58:25,228089 ][ I ][ wordplay / trainer : 734 ] Saving model to: /Users/samforeman/projects/saforem2/intro-hpc-bootcamp-2025/content/02-llms/7-shakespeare-example/ model.pth

[ 2025-07-23 08:58:25,327605 ][ I ][ wordplay / configs : 141 ] Appending /Users/samforeman/projects/saforem2/intro-hpc-bootcamp-2025/content/02-llms/ 7-shakespeare-example to /Users/samforeman/projects/saforem2/wordplay/src/ckpts/ checkpoints.log

...

[ 2025-07-23 08:59:24,391195 ][ I ][ wordplay / trainer : 850 ] step = 960 loss = 4.225 dt = 0.0204505 dtf = 0.0197843 dtb = 0.000154666 sps = 48.8985 sps_per_gpu = 48.8985 tps = 6259 tps_per_gpu = 6259 mfu = 0.128358

[ 2025-07-23 08:59:24,588273 ][ I ][ wordplay / trainer : 850 ] step = 970 loss = 4.17741 dt = 0.0188218 dtf = 0.018393 dtb = 0.000156083 sps = 53.1299 sps_per_gpu = 53.1299 tps = 6800.63 tps_per_gpu = 6800.63 mfu = 0.130217

[ 2025-07-23 08:59:24,786397 ][ I ][ wordplay / trainer : 850 ] step = 980 loss = 4.1707 dt = 0.0187595 dtf = 0.0184091 dtb = 0.000133 sps = 53.3063 sps_per_gpu = 53.3063 tps = 6823.21 tps_per_gpu = 6823.21 mfu = 0.13194

[ 2025-07-23 08:59:25,033484 ][ I ][ wordplay / trainer : 850 ] step = 990 loss = 4.1891 dt = 0.0312155 dtf = 0.0285192 dtb = 0.000297542 sps = 32.0354 sps_per_gpu = 32.0354 tps = 4100.53 tps_per_gpu = 4100.53 mfu = 0.127607

[ 2025-07-23 08:59:25,427635 ][ I ][ wordplay / trainer : 850 ] step = 1000 loss = 4.2423 dt = 0.0245103 dtf = 0.0234686 dtb = 0.000533542 sps = 40.7992 sps_per_gpu = 40.7992 tps = 5222.3 tps_per_gpu = 5222.3 mfu = 0.126131

## Evaluate Model

In [12]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=2,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-23 08:59:27,718951 ][ I ][ ipykernel_34035 / 582817405 : 12 : ezpz.log ] took: 1. 9260s

[ 2025-07-23 08:59:27,724831 ][ I ][ ipykernel_34035 / 582817405 : 13 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 08:59:27,726479 ][ I ][ ipykernel_34035 / 582817405 : 14 : ezpz.log ] [ 'response' ] :

What is an LLM?ZxxA--- 'aaaaeeewAAAAA''' qqqqqqqqqqqqaeeqqqqqq' '333qqAAA33akkk''qqqqqorrrrrrrrrrqqqqqqq.qe333aaaqqqqqf..qqqqqqq3333333-qqqqbbb''ggSSpMMMqqqqMMqqqqqqqqWW;?;?;?;???;;??MMMM;;;;;;??;;;;;;;;''''';??qqqqqqqW;;''''''''''''' ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'tttttMM